In [ ]:
import pandas as pd
import numpy as np

path = "E:\DATA ANALYST INTERN\DATASET\marketing_campaign.csv"

df = pd.read_csv(path, sep="\t", encoding="utf-8", low_memory=False)
if df.shape[1] == 1:
    
    df = pd.read_csv(path, sep=",", encoding="utf-8", low_memory=False)

print("Loaded shape:", df.shape)
df.head()


Loaded shape: (2240, 29)


<>:4: SyntaxWarning: invalid escape sequence '\D'
<>:4: SyntaxWarning: invalid escape sequence '\D'
C:\Users\ashik\AppData\Local\Temp\ipykernel_17856\2204968702.py:4: SyntaxWarning: invalid escape sequence '\D'
  path = "E:\DATA ANALYST INTERN\DATASET\marketing_campaign.csv"


,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [10]:
orig_cols = df.columns.tolist()
df.columns = [str(c).strip().lower().replace(" ", "_").replace("-", "_").replace("?", "").replace("/", "_").replace("\\", "_") for c in df.columns]
print("Before -> After (first 10):")
for o, n in zip(orig_cols[:10], df.columns[:10]):
    print(o, "->", n)


Before -> After (first 10):
ID -> id
Year_Birth -> year_birth
Education -> education
Marital_Status -> marital_status
Income -> income
Kidhome -> kidhome
Teenhome -> teenhome
Dt_Customer -> dt_customer
Recency -> recency
MntWines -> mntwines


In [11]:
if "dt_customer" in df.columns:
    df["dt_customer"] = pd.to_datetime(df["dt_customer"], errors="coerce", dayfirst=True)
    print("dt_customer nulls:", df["dt_customer"].isnull().sum())


dt_customer nulls: 0


In [12]:
maybe_numeric = []
for col in df.columns:
    s = df[col].astype(str).str.replace(",", "", regex=False).str.replace("$", "", regex=False).str.strip()
    temp = pd.to_numeric(s, errors='coerce')
    non_null_ratio = temp.notnull().sum() / len(temp)
    if non_null_ratio >= 0.7:
        maybe_numeric.append(col)
        df[col] = temp

print("Converted columns ->", maybe_numeric)


Converted columns -> ['id', 'year_birth', 'income', 'kidhome', 'teenhome', 'recency', 'mntwines', 'mntfruits', 'mntmeatproducts', 'mntfishproducts', 'mntsweetproducts', 'mntgoldprods', 'numdealspurchases', 'numwebpurchases', 'numcatalogpurchases', 'numstorepurchases', 'numwebvisitsmonth', 'acceptedcmp3', 'acceptedcmp4', 'acceptedcmp5', 'acceptedcmp1', 'acceptedcmp2', 'complain', 'z_costcontact', 'z_revenue', 'response']


In [13]:
# year_birth -> age
if "year_birth" in df.columns:
    df["year_birth"] = pd.to_numeric(df["year_birth"], errors="coerce").astype("Int64")
    df["age"] = 2025 - df["year_birth"]

# kidhome/teenhome
for c in ("kidhome", "teenhome"):
    if c in df.columns:
        df[c] = pd.to_numeric(df[c], errors="coerce").fillna(0).astype(int)

# tidy education/marital_status
for c in ("education", "marital_status"):
    if c in df.columns:
        df[c] = df[c].astype(str).str.strip().replace({"nan":"Unknown","None":"Unknown"})
        df[c] = df[c].where(df[c] != "Unknown", np.nan)
        df[c] = df[c].str.title()


In [14]:
before = len(df)
if "id" in df.columns:
    df = df.drop_duplicates(subset=["id"])
else:
    df = df.drop_duplicates()
print("Dropped duplicates:", before - len(df))


Dropped duplicates: 0


In [15]:
print("Missing BEFORE:\n", df.isnull().sum().sort_values(ascending=False).head(20))

# Numeric impute (median)
num_cols = df.select_dtypes(include=[np.number]).columns.tolist()
for c in num_cols:
    if df[c].isnull().any():
        df[c].fillna(df[c].median(), inplace=True)

# Categorical impute (mode or 'Unknown')
cat_cols = df.select_dtypes(include=["object"]).columns.tolist()
for c in cat_cols:
    if df[c].isnull().any():
        m = df[c].mode()
        if len(m)>0:
            df[c].fillna(m[0], inplace=True)
        else:
            df[c].fillna("Unknown", inplace=True)

print("Missing AFTER:\n", df.isnull().sum().sort_values(ascending=False).head(20))


Missing BEFORE:
 income                 24
id                      0
year_birth              0
education               0
marital_status          0
kidhome                 0
teenhome                0
dt_customer             0
recency                 0
mntwines                0
mntfruits               0
mntmeatproducts         0
mntfishproducts         0
mntsweetproducts        0
mntgoldprods            0
numdealspurchases       0
numwebpurchases         0
numcatalogpurchases     0
numstorepurchases       0
numwebvisitsmonth       0
dtype: int64
Missing AFTER:
 id                     0
year_birth             0
education              0
marital_status         0
income                 0
kidhome                0
teenhome               0
dt_customer            0
recency                0
mntwines               0
mntfruits              0
mntmeatproducts        0
mntfishproducts        0
mntsweetproducts       0
mntgoldprods           0
numdealspurchases      0
numwebpurchases        0
numcatalo

C:\Users\ashik\AppData\Local\Temp\ipykernel_17856\3775088557.py:7: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df[c].fillna(df[c].median(), inplace=True)


In [19]:
out_path = "E:\DATA ANALYST INTERN\CLEANED\marketingfinal_campaign.csv"
df.to_csv(out_path, index=False)
print("Saved cleaned file to:", out_path)


Saved cleaned file to: E:\DATA ANALYST INTERN\CLEANED\marketingfinal_campaign.csv


<>:1: SyntaxWarning: invalid escape sequence '\D'
<>:1: SyntaxWarning: invalid escape sequence '\D'
C:\Users\ashik\AppData\Local\Temp\ipykernel_17856\1208848529.py:1: SyntaxWarning: invalid escape sequence '\D'
  out_path = "E:\DATA ANALYST INTERN\CLEANED\marketingfinal_campaign.csv"
